In [7]:
import pandas as pd
import numpy as np
import json
import os
import re
from tqdm.notebook import tqdm
tqdm.pandas()
import sklearn.feature_extraction
#import itertools, string, operator, re, unicodedata, nltk
#from nltk.corpus import wordnet
#from nltk.stem import WordNetLemmatizer
#from nltk.tokenize import RegexpTokenizer
#import nltk
#from nltk.corpus import stopwords
'''Features'''
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
'''Classifiers'''
from sklearn.ensemble import RandomForestClassifier
'''Metrics/Evaluation'''
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
import warnings
warnings.simplefilter(action='ignore')
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jghosh2/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jghosh2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/jghosh2/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/jghosh2/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [24]:
#df = pd.read_csv('Taxml_Integers_Verified_17-11.csv',lineterminator='\n',header=0,index_col=None,usecols=['item_name','description','establishment_type','cat_name_primary','cat_name_secondary'],dtype={'item_name': str, 'description': str,'establishment_type': str, 'cat_name_primary':str,'cat_name_secondary':str})
#df = pd.read_excel('Taxml_Integers_Verified_17-11.xlsx',header=1,usecols=['item_name','description','establishment_type','cat_name_primary','cat_name_secondary'])
df = pd.read_csv('taxml_data_17_11.csv', encoding='utf8',engine='python',usecols=['item_name','description','establishment_type','CAT_Name_combined','cat_name_primary','cat_name_secondary','cat_multilevel'])
#df=df.dropna()

In [25]:
#new_stopwords=['grocery']
stpwrd = nltk.corpus.stopwords.words('english')
#stpwrd.extend(new_stopwords)

In [26]:
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stpwrd]
    lemma_words=[lemmatizer.lemmatize(w) for w in filtered_words]
    final_text=" ".join([w for w in lemma_words])
    return final_text
def cat_name_sorted(text):
    text=sorted(text.split(','))
    text=','.join([x for x in text])
    return text
def cat_name_without_temp(text):
    if ',TEMP_HEATED' in text:
        text=re.sub(',TEMP_HEATED','', text)
    if ',TEMP_UNHEATED' in text:
        text=re.sub(',TEMP_UNHEATED','', text)
    if ',TEMP_COLD' in text:
        text=re.sub(',TEMP_COLD','', text)
    if 'TEMP_HEATED' in text:
        text=re.sub('TEMP_HEATED','', text)
    if 'TEMP_UNHEATED' in text:
        text=re.sub('TEMP_UNHEATED','', text)
    if 'TEMP_COLD' in text:
        text=re.sub('TEMP_COLD','', text)
    else:
        pass
    return text



In [160]:
df.shape

(169952, 7)

In [159]:
#df1=df.sample(n=2000,random_state=42)
df1=df.sample(frac=1, random_state=42)
#df1['CAT_Name']=df1['CAT_Name'].progress_apply(lambda x: cat_name_sorted(x))
#df1['CAT_Name']=df1['CAT_Name'].progress_apply(lambda x: cat_name_without_temp(x))
df1=df1.replace(r'^\s*$', np.nan, regex=True)
df1 = df1.dropna(subset=['cat_multilevel'])
df1=df1.drop_duplicates(subset=['item_name','description','establishment_type','cat_multilevel'],ignore_index=True,keep=False)
df1['input_str'] = df1[['item_name', 'description','establishment_type']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
df1['cleanText']=df1['input_str'].map(lambda s:preprocess(s)) 
df1=df1.drop_duplicates(subset=['cat_multilevel','cleanText'],ignore_index=True,keep='first')
df1 = df1.reset_index(drop=True)

In [161]:
df1['cat_multilevel_count'] = df1.groupby('cat_multilevel')['cat_multilevel'].transform('count')
df2=df1[df1['cat_multilevel_count']<2].reset_index()
df3=df1[df1['cat_multilevel_count']>2].reset_index()
X=df3[['item_name','description','establishment_type','cleanText','cat_multilevel']]
Y=df3['cat_multilevel']

In [162]:
#Train test split with stratified sampling for evaluation
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    Y,
                                                    test_size = .20, 
                                                    random_state = 42,
                                                    stratify=Y)

In [163]:
X_train_final=pd.concat([X_train['cleanText'], df2['cleanText']])
y_train_final=pd.concat([y_train, df2['cat_multilevel']])
X_train_final=X_train_final.values
y_train_final=y_train_final.values

In [164]:
c=[]
for i in y_train_final:
    v=i.split(',')
    c.append(v)


In [165]:
d=[]
for i in y_test:
    v=i.split(',')
    d.append(v)

In [166]:
#from sklearn.naive_bayes import MultinomialNB

from sklearn.multioutput import MultiOutputClassifier
rf = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', RandomForestClassifier()),
              ])

n_classes = 2
multi_target_forest = MultiOutputClassifier(rf, n_jobs=-1)
multi_target_forest.fit(X_train_final, np.array(c))
y_pred=multi_target_forest.predict(X_test['cleanText'])
#rf.fit(X_train_final, y_train_final)



In [167]:
multi_target_forest.score(X_train_final, np.array(c))

0.9681797092483762

In [168]:
new_series_pred = pd.DataFrame(y_pred,columns=['c', 'a'])
new_series_pred['pred']=new_series_pred['c']+','+new_series_pred['a']
new_series_test= pd.DataFrame(np.array(d),columns=['c', 'a'])
new_series_test['test']=new_series_test['c']+','+new_series_test['a']

In [169]:
new_series=pd.concat([new_series_pred, new_series_test], axis=1)

In [170]:
new_series.to_csv('res.csv')

In [171]:
new_series
misclassifications = new_series.loc[new_series['test']!=new_series['pred']]

In [172]:
misclassifications.shape

(6411, 6)

In [174]:
misclassifications.to_csv('mis.csv')

In [175]:
print(accuracy_score(new_series['pred'],new_series['test']))
print(precision_score(new_series['pred'],new_series['test'],average='macro'))
print(recall_score(new_series['pred'],new_series['test'],average='macro'))
print(f1_score(new_series['pred'],new_series['test'],average='macro'))

0.7521073389529038
0.1303058013315739
0.17027339787807286
0.14353636961732968


In [86]:
y_pred = rf.predict(X_test['cleanText'])
print(f'accuracy {accuracy_score(y_pred,y_test)}')
print(precision_score(y_pred,y_test,average='macro'))
print(recall_score(y_pred,y_test,average='macro'))
print(f1_score(y_pred,y_test,average='macro'))

accuracy 0.77971149011873
0.605444931270198
0.7025870291816179
0.6316093970912848


In [90]:
result=pd.DataFrame()
result=X_test
result['original_cat']=y_test
result['pred']=y_pred
misclassifications = result.loc[result['original_cat']!=result['pred']]
print(misclassifications.shape)
print(result.shape)
misclassifications.to_excel('misclassifications_sample.xlsx')

(5696, 6)
(25857, 6)


In [91]:

with pd.ExcelWriter('misclassifications_sample.xlsx') as writer:  
    for i,label in enumerate(misclassifications['original_cat']):
        df = misclassifications.loc[misclassifications['original_cat'] == label]
        df.to_excel(writer, sheet_name='{}'.format(label[:30]))

In [66]:
import pickle
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(rf, open(filename, 'wb'))
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)


In [67]:
#Train test split with stratified sampling for evaluation
X_train, x_val, y_train, y_val = train_test_split(df3['input_str'],
                                                    df3['CAT_Name'],
                                                    test_size = .4, 
                                                    random_state = 42,
                                                    stratify=df3['CAT_Name'])

In [68]:
print(loaded_model.score(x_val, y_val))

0.8273388250763817


In [ ]:
with pd.ExcelWriter('misclassifications_check.xlsx') as writer:  
    for i,label in enumerate(misclassifications):
        df = misclassifications.loc[result['CAT_Name'] == label]
        df.to_excel(writer, sheet_name='Sheet_name_{}'.format(i))

In [76]:
 for i,label in enumerate(misclassifications):
        print(i,label)

0 cleanText
1 original_cat
2 pred


In [75]:
misclassifications

,cleanText,original_cat,pred
56934,celsius pear,"CAT_ENERGY_DRINK,CONTAINER_BOTTLED",CAT_JUICE_NON_CARBONATED
7532,real kidney bean,CAT_PREPACKAGED_FOOD_SHELF_STABLE_POTATOES,CAT_PREPACKAGED_FOOD
117757,david jumbo dill pickle sunflower seed,CAT_PREPACKAGED_FOOD_SNACK_NUTS,CAT_SEEDS
41013,coca cola zero liter,CAT_ALCOHOL,"CAT_SOFT_DRINK,CONTAINER_BOTTLED"
55272,mdh meat masala mdh meat masala,CAT_PREPACKAGED_FOOD_SEASONING,CAT_PREPACKAGED_FOOD
...,...,...,...
87673,devacurl leave miracle curl plumper devacurl b...,CAT_TPP_SHAMPOOS,CAT_TPP
3977,julio blanco liquor bottle abv,CAT_ALCOHOL,CAT_LIQUOR
95590,planet oregon fiz sparkling rose flower,CAT_SPARKLING_WINE,CAT_TPP
49560,patchology perfect ten,CAT_TPP_BODY_LOTION,CAT_TPP_SKIN_CARE_PRODUCTS


In [123]:

model_dict = {'Random Forest': Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('rf', RandomForestClassifier()),
              ]),
             
             'naive bayas': Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ]),
              'logistic': Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('lg', LogisticRegression()),
              ])
             }
            

#Function to get the scores for each model in a df
def model_score_df(model_dict):   
    model_name, ac_score_list, p_score_list, r_score_list, f1_score_list = [], [], [], [], []
    for k,v in tqdm(model_dict.items()):   
        model_name.append(k)
        v.fit(X_train_final, y_train_final)
        y_pred = v.predict(X_test)
        ac_score_list.append(accuracy_score(y_test, y_pred))
        p_score_list.append(precision_score(y_test, y_pred, average='macro'))
        r_score_list.append(recall_score(y_test, y_pred, average='macro'))
        f1_score_list.append(f1_score(y_test, y_pred, average='macro'))
        model_comparison_df = pd.DataFrame([model_name, ac_score_list, p_score_list, r_score_list, f1_score_list]).T
        model_comparison_df.columns = ['model_name', 'accuracy_score', 'precision_score', 'recall_score', 'f1_score']
        model_comparison_df = model_comparison_df.sort_values(by='f1_score', ascending=False)
    return model_comparison_df


In [124]:
print(model_score_df(model_dict))

  0%|          | 0/3 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
#################redundent code###################

In [ ]:
X_train

In [ ]:
#Creating the features (tf-idf weights) for the processed text

#texts = df1['input_str'].astype('str')

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2),max_df = .95)
LE = LabelEncoder()
#tfidf
tfv = TfidfVectorizer(strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',ngram_range=(1,2), use_idf=1,smooth_idf=1,sublinear_tf=1,max_df = .95,stop_words = 'english')

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train) #features
X_test_tfidf= tfidf_vectorizer.fit_transform(X_test) #features

tfv.fit(list(X_train) + list(X_test))
xtrain_tfv =  tfv.transform(X_train) 
xvalid_tfv = tfv.transform(X_test)

#countvec
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')
X_train_ctv = ctv.fit_transform(X_train) #features
X_test_ctv= ctv.fit_transform(X_test) #features



y_train_final=LE.fit_transform(y_train)
y_test_final=LE.fit_transform(y_test)

In [ ]:

#Dimenionality reduction. Only using the 100 best features er category

lsa = TruncatedSVD(n_components=100, 
                   n_iter=10, 
                   random_state=3)

X_train_final = lsa.fit_transform(X_train_tfidf)
X_test_final = lsa.fit_transform(X_test_tfidf)

X_train_final_ctv= lsa.fit_transform(X_train_ctv)
X_test_final_ctv= lsa.fit_transform(X_test_ctv)

In [ ]:
y_pred

In [ ]:
model_name, ac_score_list, p_score_list, r_score_list, f1_score_list = [], [], [], [], []
randomforestmodel=RandomForestClassifier(random_state=3)
#tfidf
#randomforestmodel.fit(xtrain_tfv, y_train_final)
#y_pred = randomforestmodel.predict(xvalid_tfv)
#countvec
#randomforestmodel.fit(X_train_final_ctv, y_train_final)
#y_pred = randomforestmodel.predict(X_test_final_ctv)
#svd
randomforestmodel.fit(X_train_final, y_train_final)
y_pred = randomforestmodel.predict(X_test_final)

ac_score_list.append(accuracy_score(y_test_final, y_pred))
p_score_list.append(precision_score(y_test_final, y_pred, average='macro'))
r_score_list.append(recall_score(y_test_final, y_pred, average='macro'))
f1_score_list.append(f1_score(y_test_final, y_pred, average='macro'))
model_comparison_df = pd.DataFrame([model_name, ac_score_list, p_score_list, r_score_list, f1_score_list]).T
model_comparison_df.columns = ['model_name', 'accuracy_score', 'precision_score', 'recall_score', 'f1_score']
model_comparison_df = model_comparison_df.sort_values(by='f1_score', ascending=False)
print(model_comparison_df)

In [ ]:

#Preliminary model evaluation using default parameters
from sklearn.naive_bayes import MultinomialNB
#Creating a dict of the models
model_dict = {'Random Forest': RandomForestClassifier(random_state=3)}
            

#Function to get the scores for each model in a df
def model_score_df(model_dict):   
    model_name, ac_score_list, p_score_list, r_score_list, f1_score_list = [], [], [], [], []
    for k,v in tqdm(model_dict.items()):   
        model_name.append(k)
        v.fit(X_train_final, y_train_final)
        y_pred = v.predict(X_test_final)
        ac_score_list.append(accuracy_score(y_test_final, y_pred))
        p_score_list.append(precision_score(y_test_final, y_pred, average='macro'))
        r_score_list.append(recall_score(y_test_final, y_pred, average='macro'))
        f1_score_list.append(f1_score(y_test_final, y_pred, average='macro'))
        model_comparison_df = pd.DataFrame([model_name, ac_score_list, p_score_list, r_score_list, f1_score_list]).T
        model_comparison_df.columns = ['model_name', 'accuracy_score', 'precision_score', 'recall_score', 'f1_score']
        model_comparison_df = model_comparison_df.sort_values(by='f1_score', ascending=False)
    return model_comparison_df,v



In [ ]:
final_dict=model_score_df(model_dict)

In [ ]:
final_dict[1]

In [ ]:
print(final_dict[0])

In [45]:
from sklearn.model_selection import RandomizedSearchCV

In [105]:
#clf = RandomForestClassifier()
from pprint import pprint
from time import time
import logging
rf=Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', RandomForestClassifier()),
              ])

parameters = {
    "vect__max_df": (0.5, 0.75, 1.0),
    # 'vect__max_features': (None, 5000, 10000, 50000),
    "vect__ngram_range": ((1, 1), (1, 2)),  # unigrams or bigramslf__
     "clf__bootstrap":[True, False],
     "clf__max_depth":[10, 50, 100,500, None],
     "clf__max_features":['auto', 'sqrt'],
     "clf__min_samples_leaf":[1,2,4],
     "clf__min_samples_split":[2,5,10],
     "clf__n_estimators":[400,600,800],
     "clf__random_state":[3]

}

RandomizedSearch = RandomizedSearchCV(rf,
                          parameters, 
                          cv=5,
                          verbose=1, 
                          n_jobs=-1)

t0 = time()
rf_best_model = RandomizedSearch.fit(X_train_final, y_train_final)
print("done in %0.3fs" % (time() - t0))
#print()
print("Best score: %0.3f" % rf_best_model.best_score_)
print("Best parameters set:")
best_parameters = rf_best_model.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))


Fitting 5 folds for each of 10 candidates, totalling 50 fits


/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


done in 27.282s
Best score: 0.592
Best parameters set:
	clf__bootstrap: True
	clf__max_depth: 100
	clf__max_features: 'sqrt'
	clf__min_samples_leaf: 1
	clf__min_samples_split: 2
	clf__n_estimators: 400
	clf__random_state: 3
	vect__max_df: 1.0
	vect__ngram_range: (1, 1)


In [103]:
rf_best_model

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                             ('tfidf', TfidfTransformer()),
                                             ('clf',
                                              RandomForestClassifier())]),
                   n_jobs=-1,
                   param_distributions={'clf__bootstrap': [True, False],
                                        'clf__max_depth': [10, 50, 100, None],
                                        'clf__max_features': ['auto', 'sqrt'],
                                        'clf__min_samples_leaf': [1, 2, 4],
                                        'clf__min_samples_split': [2, 5, 10],
                                        'clf__n_estimators': [800, 1400, 2000],
                                        'clf__random_state': [3],
                                        'vect__max_df': (0.5, 0.75, 1.0),
                                        'vect__ngram_range': (

In [ ]:
rf_best_model = RandomForestClassifier(bootstrap = False,
                                       max_depth = 50,
                                       max_features = 'auto',
                                       min_samples_leaf = 1,
                                       n_estimators = 1400,
                                       random_state=3)

In [ ]:
ac_score_list, p_score_list, r_score_list, f1_score_list = [], [], [], []
rf_best_model.fit(X_train, y_train)
y_pred = rf_best_model.predict(X_test)
ac_score_list.append(accuracy_score(y_test, y_pred))
p_score_list.append(precision_score(y_test, y_pred, average='macro'))
r_score_list.append(recall_score(y_test, y_pred, average='macro'))
f1_score_list.append(f1_score(y_test, y_pred, average='macro'))
model_comparison_df = pd.DataFrame([ac_score_list, p_score_list, r_score_list, f1_score_list]).T
model_comparison_df.columns = ['accuracy_score', 'precision_score', 'recall_score', 'f1_score']
model_comparison_df = model_comparison_df.sort_values(by='f1_score', ascending=False)

In [ ]:
model_comparison_df

In [ ]:
filename = './output/finalized_model.sav'
joblib.dump(model, filename)
 
# some time later...
 
# load the model from disk
loaded_model = joblib.load(filename)
result = loaded_model.score(X_test, Y_test)
print(result)